In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-25 10:39:41.485526: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 10:39:41.548137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 10:39:41.548185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 10:39:41.549347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alre

In [2]:
df = pd.read_csv("final.csv")

In [3]:
df_subset = df.sample(n=20000, random_state=42)

In [10]:



X = df_subset['sentence_spacy'].astype(str)
y = df_subset['SentimentLabel']
max_words = 10000
maxlen = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(sequences, maxlen=maxlen)

label_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2, 'VeryNegative': 3, 'VeryPositive': 4}
y = y.map(label_dict)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(0.2))  
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=64, activation='relu')) 
model.add(Dense(units=5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy*100)


Epoch 1/10
400/400 [==============================] - 51s 125ms/step - loss: 1.2498 - accuracy: 0.5011 - val_loss: 1.0868 - val_accuracy: 0.5947
Epoch 2/10
400/400 [==============================] - 43s 106ms/step - loss: 0.8677 - accuracy: 0.6809 - val_loss: 0.8779 - val_accuracy: 0.6681
Epoch 3/10
400/400 [==============================] - 41s 102ms/step - loss: 0.5878 - accuracy: 0.7838 - val_loss: 0.7958 - val_accuracy: 0.6963
Epoch 4/10
400/400 [==============================] - 41s 102ms/step - loss: 0.4376 - accuracy: 0.8442 - val_loss: 0.7693 - val_accuracy: 0.7287
Epoch 5/10
400/400 [==============================] - 41s 103ms/step - loss: 0.3399 - accuracy: 0.8827 - val_loss: 0.8958 - val_accuracy: 0.7225
Epoch 6/10
400/400 [==============================] - 49s 123ms/step - loss: 0.2751 - accuracy: 0.9063 - val_loss: 0.8904 - val_accuracy: 0.7400
Epoch 7/10
400/400 [==============================] - 47s 119ms/step - loss: 0.2284 - accuracy: 0.9245 - val_loss: 0.9946 - val_ac

In [12]:



X = df_subset['sentence_spacy'].astype(str)
y = df_subset['SentimentLabel']

vectorizer = CountVectorizer(max_features=10000)
X_vectorized = vectorizer.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy * 100)


Test Accuracy: 54.125
